In [5]:
import praw
import os
import configparser
import webbrowser
import random
import json
import pandas as pd
import numpy as np
import collections
import string
from nltk.corpus import stopwords
from nltk.stem import *
os.chdir("C:/Users/macle/Desktop/UPC Masters/Semester 2/CI/Final Project")

In [6]:
config = configparser.ConfigParser()
config.read('secrets.ini')

reddit_client_id = config.get('reddit', 'client_id')
reddit_api_key = config.get('reddit', 'api_key')

r = praw.Reddit('reddit recommender by u/upcmaici ver 0.0.1 ')
r.set_oauth_app_info(client_id=reddit_client_id,
                      client_secret=reddit_api_key,
                      redirect_uri='http://cole-maclean.github.io/MAI-CI/')

In [7]:
url = r.get_authorize_url('uniqueKey', 'identity', True)
webbrowser.open(url)

True

In [4]:
access_information = r.get_access_information('NRQP3Xn0A5sYd-pLj4wxVvnn6Xo')
r.set_access_credentials(**access_information)

In [5]:
authenticated_user = r.get_me()
print(authenticated_user.name, authenticated_user.link_karma)

upcmaici 1


In [4]:
with open('scrapped_users.json','r') as data_file:    
    user_data = json.load(data_file)
len(user_data)

58

In [112]:
translate_table = dict((ord(char), None) for char in string.punctuation)
stop = set(stopwords.words('english'))
stemmer = PorterStemmer()
with open('reddit_data.json','r') as data_file:    
    reddit_data = json.load(data_file)
with open('scrapped_users.json','r') as data_file:    
    scrapped_users = json.load(data_file)
for i in range(10):
    print ("scrapping " + str(i) + "th subreddit")
    rand_submission = r.get_random_submission()
    print ("scrapping users from subreddit r/" + rand_submission.subreddit.display_name)
    sub_comments = rand_submission.comments
    if len(sub_comments) >=3:
        rnd_comments = random.sample(sub_comments,min(10,len(sub_comments)))
        for comment in rnd_comments:
            if isinstance(comment, praw.objects.Comment):
                user = comment.author
                if user:
                    if user.name in scrapped_users:
                        print ('user ' + user.name + 'already scraped')
                    else:
                        scrapped_users.append(user.name)
                        for user_comment in user.get_comments(limit=None,_use_oauth=False):
                            body = user_comment.body.split()
                            if len(body) >= 10:
                                rand_words = random.sample(body,10)
                            else:
                                rand_words = body
                            clean_comment_words = []
                            for word in rand_words:
                                if len(word) < 45 and word not in stop:
                                    clean_word = word.translate(translate_table)
                                    wrd_stm = stemmer.stem(clean_word)
                                    clean_comment_words.append(wrd_stm)
                            reddit_data.append([user.name,user_comment.subreddit.display_name,user_comment.link_title.split(' '),
                                                                              user_comment.created_utc,clean_comment_words])
                    
with open('reddit_data.json','w') as data_file:
    json.dump(reddit_data, data_file)
with open('scrapped_users.json','w') as data_file:
    json.dump(scrapped_users, data_file)

scrapping 0th subreddit
scrapping users from subreddit r/ImGoingToHellForThis
scrapping 1th subreddit
scrapping users from subreddit r/Marvel
scrapping 2th subreddit
scrapping users from subreddit r/LateStageCapitalism
scrapping 3th subreddit
scrapping users from subreddit r/happygirls
scrapping 4th subreddit
scrapping users from subreddit r/bidenbro
scrapping 5th subreddit
scrapping users from subreddit r/aww
scrapping 6th subreddit
scrapping users from subreddit r/chicagobulls
scrapping 7th subreddit
scrapping users from subreddit r/politics
scrapping 8th subreddit
scrapping users from subreddit r/ANormalDayInRussia
scrapping 9th subreddit
scrapping users from subreddit r/exxxchange


In [113]:
list_of_words = []
translate_table = dict((ord(char), None) for char in string.punctuation)
stop = set(stopwords.words('english'))
stemmer = PorterStemmer()
for user_comment in reddit_data:
    for word in user_comment[4]:
        if len(word) < 45 and word not in stop:
            clean_word = word.translate(translate_table)
            wrd_stm = stemmer.stem(clean_word)
            list_of_words.append(wrd_stm)
word_counts = collections.Counter(list_of_words)

In [117]:
df = pd.DataFrame(reddit_data,columns=['user','subreddit','submission','utc_stamp','rnd_words'])
df.head(25)

,user,subreddit,submission,utc_stamp,rnd_words
0,popesnutsack,worldpolitics,[102 million dead California trees 'unpreceden...,1479651635,"[water, If, one, tree]"
1,popesnutsack,Offensive_Wallpapers,[Ya gotta love PornHub comments [2400 x 1350]],1479651372,"[My, next, alimoni, payment]"
2,popesnutsack,funny,"[""Oh my fucking god!""]",1479649683,"[Now, hard, on]"
3,popesnutsack,ImGoingToHellForThis,[First Day in Office],1479517797,"[Can, hope, cheeto, board]"
4,popesnutsack,mildlyinteresting,[We do Not hire LIBERALS. Rude sign in conveni...,1479512926,"[Veri, eloqu, craft, wordsmith]"
5,popesnutsack,AskReddit,"[Hey Reddit, what's your biggest ""We can put a...",1479504649,"[We, differenti, anal, moon, oral]"
6,popesnutsack,worldpolitics,[Liberals are RACIST ?],1479500264,"[Thi, rfacepalm]"
7,popesnutsack,AskReddit,"[The world is taken over by ducks, and humanki...",1479500111,"[Invest, stale, bread, market]"
8,popesnutsack,AskReddit,[What is the best way to cook a turkey for Tha...,1479499936,"[Buy, stove, box, follow, direct]"
9,popesnutsack,WTF,[Jonestown-Massacre 1978: Over 900 people of a...,1479499847,"[punchlin, rememb, joke, dont, there]"


In [118]:
f = {'subreddit':['count'], 'utc_stamp':[np.min,np.max],'submission': 'sum','rnd_words': 'sum'}
grouped = df.groupby(['user','subreddit']).agg(f)

In [119]:
grouped

utc_stamp             subreddit  \
                                             amin        amax     count   
user             subreddit                                                
BlackToothBandit 49ers                 1469412928  1475093452        13   
                 AskReddit             1469997014  1478429480        22   
                 BlackPeopleTwitter    1470272686  1470272686         1   
                 Comcast_Xfinity       1476183035  1476183035         1   
                 Defenders             1475164307  1476440370         2   
                 EnoughTrumpSpam       1476939749  1476940564         2   
                 FearTheWalkingDead    1472699552  1474440398         2   
                 FlashTV               1463237583  1478963476         4   
                 GreenBayPackers       1474435813  1474435813         1   
                 LegendsOfTomorrow     1463525875  1472533809         4   
                 MadeMeSmile           1474520552  1474520552         1   
                 Marvel                1472350014  1479673782         4   
                 MensRights            1475089736  1475089736         1   
                 PS4                   1476179008  1476179008         1   
                 Preacher              1469699361  1470183083         5   
                 Showerthoughts        1474005446  1474005446         1   
                 TheStrain             1472495647  1476255225         6   
                 WTF                   1472552280  1476238525         3   
                 asoiaf                1470568778  1475380554         2   
                 comicbookmovies       1476060566  1476060566         1   
                 conspiracy            1474769454  1477043348         6   
                 explainlikeimfive     1470223674  1472426301         2   
                 freefolk              1476060372  1476060372         1   
                 funny                 1473490961  1473490961         1   
                 future_fight          1474338997  1474339166         2   
                 gameofthrones         1474253510  1475110021         6   
                 gifs                  1475043341  1475203701         6   
                 marvelstudios         1472396009  1474476639         2   
                 mildlyinfuriating     1474499191  1474499191         1   
                 nfl                   1466347192  1476254628        10   
...                                           ...         ...       ...   
vexion           smashbros             1422778381  1422784371         2   
                 space                 1427121120  1427121120         1   
                 standupshots          1376426713  1405002595         2   
                 starcitizen           1426997562  1426997562         1   
                 starcraft             1315071150  1326737664        28   
                 streetlightmanifesto  1372115851  1459186324         7   
                 talesfromtechsupport  1399308066  1423780403         5   
                 talesfromthejob       1423796331  1423796331         1   
                 talesfromthelaw       1399646646  1452609487        15   
                 technology            1354149919  1449170071         4   
                 techsupport           1380140094  1473886550         6   
                 television            1360973866  1421290197         2   
                 tf2                   1320004357  1320004357         1   
                 tf2trade              1320113645  1320113645         1   
                 thewalkingdead        1425879033  1447034905         3   
                 todayilearned         1315598113  1397085719        12   
                 trailers              1427035954  1427035954         1   
                 trashy                1462027942  1462027942         1   
                 triathlon             1447097770  1447097770         1   
                 trippinthroughtime    1444693115  1444693115    